In [1]:
#!pip install scikit-learn --force
#!pip install catboost

In [2]:
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd
from datetime import datetime
import os
import importlib

pd.set_option('display.max.columns', 300)

from core.calculator.storage import ModelDB
from core.calculator.core import ForecastConfig
from core.calculator.core import TrainingManager
from core.calculator.core import ForecastConfig
from core.calculator.core import ForecastEngine

from core.calculator.deposits import DepositsCalculationType
from core.calculator.deposits import DepositIterativeCalculator

from core.definitions import *
from core.project_update import load_portfolio

from core.models import DepositModels
from warnings import filterwarnings
filterwarnings('ignore')

### Подготовка таблицы сценария

In [3]:
# Дата из который мы прогнозируем (пока что не меняется)
train_end = datetime(year=2022, month=12, day=31)

# Горизонт прогноза в месяцах
horizon = 12

In [4]:
# Данные для прогноза
scenario_data = {
    # Ожидаемый баланс на первую дату прогноза, задавать необязательно
     'expected_amount':      [np.nan for h in range(horizon)],
    # ССВ
     'SSV':                  [0.48 for h in range(horizon)],
    # ФОР
     'FOR':                  [4.5 for h in range(horizon)],
    # Трансфертные ставки
     'VTB_ftp_rate_[90d]':   [12.3 for h in range(horizon)],
     'VTB_ftp_rate_[180d]':  [12 for h in range(horizon)],
     'VTB_ftp_rate_[365d]':  [12 for h in range(horizon)],
     'VTB_ftp_rate_[548d]':  [12 for h in range(horizon)],
     'VTB_ftp_rate_[730d]':  [12 for h in range(horizon)],
     'VTB_ftp_rate_[1095d]': [12 for h in range(horizon)],
    
    # Маржа бизнеса по срочностям
     'margin_[90d]':         [0.1 for h in range(horizon)],
     'margin_[180d]':        [0.1 for h in range(horizon)],
     'margin_[365d]':        [0.1 for h in range(horizon)],
     'margin_[548d]':        [0.1 for h in range(horizon)],
     'margin_[730d]':        [0.2 for h in range(horizon)],
     'margin_[1095d]':       [0.2 for h in range(horizon)],
    
    # Спред Привилегия - Массовый (на сколько в среднем ставки по сегменту Привилегия больше чем ставки по массовому сегменту)
     'priv_spread':          [0.4 for h in range(horizon)],
    # Спред ВИП - Массовый (на сколько в среднем ставки по сегменту ВИП больше чем ставки по массовому сегменту)
     'vip_spread':           [0.8 for h in range(horizon)],
    
    # Ниже три спреда по разным типам опциональности по отношению к безопциональным вкладам (Подразумевается, что они, как правило, отрицательные)
    # r - возможности пополнения, s - возможность расходных операций
    
    # На сколько ставка по расходным вкладам выше чем ставка по безопциональным вкладам (Если ниже - то со знаком минус)
     'r0s1_spread':          [-1 for h in range(horizon)],
    
    # На сколько ставка по пополняемым вкладам выше чем ставка по безопциональным вкладам (Если ниже - то со знаком минус)
     'r1s0_spread':          [-1 for h in range(horizon)],
    
    # На сколько ставка по расходно-пополняемым вкладам выше чем ставка по безопциональным вкладам (Если ниже - то со знаком минус)
     'r1s1_spread':          [-1.2 for h in range(horizon)],
    
    # Ставка по лучшему предложению сбера
     'SBER_max_rate':        [11.2 for h in range(horizon)],
    
    # Базовая ставка по НС
     'SA_rate':              [5 for h in range(horizon)]
}
scenario_df_user = pd.DataFrame(scenario_data)

In [5]:
scenario_df = preprocess_scenario(scenario_df_user, train_end, horizon)

In [6]:
tmp_ind = scenario_df.index

In [7]:
sc_data = pd.read_csv('scenario_table.csv')

In [8]:
col_list = sc_data.columns.copy()

d = {}
for col in col_list: 
    
    new_name = col[26:]
    
    d[col] = new_name

In [9]:
sc_data.columns = d.values()

In [10]:
#scenario_df.columns = scenario_df.columns.str.lower()

In [11]:
scenario_df.iloc[:,1:].shape

(12, 80)

In [12]:
sc_data.head(1).shape

(1, 81)

In [13]:
d1 = {}

col_list1  = scenario_df.columns

for col in col_list1: 
    
    new_name = col.lower()
    
    d1[new_name] = col

In [14]:
sc_data.rename(d1, inplace = True)

In [15]:
sc_data['expected_amount'] = np.nan

In [16]:
sc_data['report_dt'] = pd.to_datetime(sc_data['report_dt'])

In [17]:
sc_data_final = sc_data[(sc_data['report_dt']>='2023-01-01')&(sc_data['report_dt']<='2023-10-01')].fillna(method='bfill').copy()

In [18]:
sc_data_final = sc_data_final.fillna(method='ffill').copy()

In [19]:
sc_data_final.index = sc_data_final['report_dt']

In [20]:
del sc_data_final['report_dt']

In [21]:
sc_data_final = sc_data_final.sort_index()

In [22]:
scenario_df1 = sc_data_final.copy()

In [23]:
scenario_df1 = scenario_df1[scenario_df.columns.str.lower()]

In [24]:
scenario_df1.columns = scenario_df.columns

In [25]:
scenario_df1.loc['2023-10-31', :] = scenario_df1.loc['2023-09-30', :]
scenario_df1.loc['2023-11-30', :] = scenario_df1.loc['2023-09-30', :]
scenario_df1.loc['2023-12-31', :] = scenario_df1.loc['2023-09-30', :]

In [26]:
scenario_df1 = scenario_df1.fillna(method='ffill')

In [27]:
nan_cols = scenario_df.columns[scenario_df1.isna().any()]
#nan_cols

In [28]:
scenario_df1 = scenario_df1.fillna(0.01)
scenario_df1['expected_amount'] = np.nan

In [29]:
scenario_df1.index = tmp_ind

### Другое

In [30]:
# scenario_df1 = sc_data_final[(sc_data_final.index>='2023-02-01')&(sc_data_final.index<='2023-05-01')]
# scenario_df1.index = scenario_df.index

In [31]:
scenario_df1

,expected_amount,SBER_max_rate,rate_sa_weighted,VTB_ftp_rate_[90d],VTB_ftp_rate_[180d],VTB_ftp_rate_[365d],VTB_ftp_rate_[548d],VTB_ftp_rate_[730d],VTB_ftp_rate_[1095d],VTB_weighted_rate_[mass]_[r0s0]_[90d],VTB_weighted_rate_[mass]_[r0s0]_[180d],VTB_weighted_rate_[mass]_[r0s0]_[365d],VTB_weighted_rate_[mass]_[r0s0]_[548d],VTB_weighted_rate_[mass]_[r0s0]_[730d],VTB_weighted_rate_[mass]_[r0s0]_[1095d],VTB_weighted_rate_[mass]_[r0s1]_[90d],VTB_weighted_rate_[mass]_[r0s1]_[180d],VTB_weighted_rate_[mass]_[r0s1]_[365d],VTB_weighted_rate_[mass]_[r0s1]_[548d],VTB_weighted_rate_[mass]_[r0s1]_[730d],VTB_weighted_rate_[mass]_[r0s1]_[1095d],VTB_weighted_rate_[mass]_[r1s0]_[90d],VTB_weighted_rate_[mass]_[r1s0]_[180d],VTB_weighted_rate_[mass]_[r1s0]_[365d],VTB_weighted_rate_[mass]_[r1s0]_[548d],VTB_weighted_rate_[mass]_[r1s0]_[730d],VTB_weighted_rate_[mass]_[r1s0]_[1095d],VTB_weighted_rate_[mass]_[r1s1]_[90d],VTB_weighted_rate_[mass]_[r1s1]_[180d],VTB_weighted_rate_[mass]_[r1s1]_[365d],VTB_weighted_rate_[mass]_[r1s1]_[548d],VTB_weighted_rate_[mass]_[r1s1]_[730d],VTB_weighted_rate_[mass]_[r1s1]_[1095d],VTB_weighted_rate_[priv]_[r0s0]_[90d],VTB_weighted_rate_[priv]_[r0s0]_[180d],VTB_weighted_rate_[priv]_[r0s0]_[365d],VTB_weighted_rate_[priv]_[r0s0]_[548d],VTB_weighted_rate_[priv]_[r0s0]_[730d],VTB_weighted_rate_[priv]_[r0s0]_[1095d],VTB_weighted_rate_[priv]_[r0s1]_[90d],VTB_weighted_rate_[priv]_[r0s1]_[180d],VTB_weighted_rate_[priv]_[r0s1]_[365d],VTB_weighted_rate_[priv]_[r0s1]_[548d],VTB_weighted_rate_[priv]_[r0s1]_[730d],VTB_weighted_rate_[priv]_[r0s1]_[1095d],VTB_weighted_rate_[priv]_[r1s0]_[90d],VTB_weighted_rate_[priv]_[r1s0]_[180d],VTB_weighted_rate_[priv]_[r1s0]_[365d],VTB_weighted_rate_[priv]_[r1s0]_[548d],VTB_weighted_rate_[priv]_[r1s0]_[730d],VTB_weighted_rate_[priv]_[r1s0]_[1095d],VTB_weighted_rate_[priv]_[r1s1]_[90d],VTB_weighted_rate_[priv]_[r1s1]_[180d],VTB_weighted_rate_[priv]_[r1s1]_[365d],VTB_weighted_rate_[priv]_[r1s1]_[548d],VTB_weighted_rate_[priv]_[r1s1]_[730d],VTB_weighted_rate_[priv]_[r1s1]_[1095d],VTB_weighted_rate_[vip]_[r0s0]_[90d],VTB_weighted_rate_[vip]_[r0s0]_[180d],VTB_weighted_rate_[vip]_[r0s0]_[365d],VTB_weighted_rate_[vip]_[r0s0]_[548d],VTB_weighted_rate_[vip]_[r0s0]_[730d],VTB_weighted_rate_[vip]_[r0s0]_[1095d],VTB_weighted_rate_[vip]_[r0s1]_[90d],VTB_weighted_rate_[vip]_[r0s1]_[180d],VTB_weighted_rate_[vip]_[r0s1]_[365d],VTB_weighted_rate_[vip]_[r0s1]_[548d],VTB_weighted_rate_[vip]_[r0s1]_[730d],VTB_weighted_rate_[vip]_[r0s1]_[1095d],VTB_weighted_rate_[vip]_[r1s0]_[90d],VTB_weighted_rate_[vip]_[r1s0]_[180d],VTB_weighted_rate_[vip]_[r1s0]_[365d],VTB_weighted_rate_[vip]_[r1s0]_[548d],VTB_weighted_rate_[vip]_[r1s0]_[730d],VTB_weighted_rate_[vip]_[r1s0]_[1095d],VTB_weighted_rate_[vip]_[r1s1]_[90d],VTB_weighted_rate_[vip]_[r1s1]_[180d],VTB_weighted_rate_[vip]_[r1s1]_[365d],VTB_weighted_rate_[vip]_[r1s1]_[548d],VTB_weighted_rate_[vip]_[r1s1]_[730d],VTB_weighted_rate_[vip]_[r1s1]_[1095d]
2023-01-31,NaN,8.000000,5.003411,8.000000,8.250000,8.550000,8.850000,9.150000,9.500000,6.999905,7.518473,7.471891,5.660102,7.578739,8.390000,0.01,0.01,0.01,0.01,0.01,0.01,5.842649,3.794318,3.268132,2.985803,2.999146,2.994811,6.029594,6.186708,5.133139,3.150000,4.504944,2.000000,7.000000,7.440814,7.551823,6.000000,7.823649,8.390000,0.01,0.01,0.01,0.01,0.01,0.01,6.552139,6.231580,5.474193,2.946051,2.924987,2.909715,6.136634,6.250000,4.750000,3.75,4.50,2.000000,7.000000,7.398181,7.620515,7.842370,8.200000,8.397031,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,6.220162,6.467566,6.175783,5.400000,5.528164,5.400000
2023-02-28,NaN,9.500000,5.047190,8.005556,8.255556,8.550000,8.850000,9.150000,9.500000,6.951340,7.439611,7.457628,5.830272,7.578739,8.390000,0.01,0.01,0.01,0.01,0.01,0.01,5.842649,3.794318,3.268132,2.985803,2.999146,2.994811,6.026958,6.326632,6.113943,3.150000,4.500000,2.000000,6.991730,7.433784,7.554141,5.113112,7.823649,8.390000,0.01,0.01,0.01,0.01,0.01,0.01,6.552139,6.231580,5.474193,2.946051,2.924987,2.909715,

### Чтение портфеля и моделей

### Посчитаем историю на каждый месяц

In [32]:
res_port = []
tmp = pd.read_csv('/home/vtb70186744/dynbalance/data/portfolio_data/portfolio_2022-12.csv')

res_port.append(tmp)


for i in range(1,10):
    path = f'/home/vtb70186744/dynbalance/data/portfolio_data/portfolio_2023-0{i}.csv'
    
    port = pd.read_csv(path)
    res_port.append(port)

In [33]:
len(res_port)

10

In [34]:
res_port[8]

,report_month,bucketed_balance,is_vip_or_prv,drop_flg,optional_flg,bucketed_period,open_month,close_month,bucketed_open_rate,report_dt,total_interests,remaining_interests,weight_rate,total_generation,total_generation_cleared,weight_renewal_cnt,weight_renewal_available_flg,weight_close_plan_day,gen_name,SER_d,SER_d_cl,SER_dinamic,SER_dinamic_cl,total_generation_lag1,total_generation_cl_lag1,CUR,report_weight_open_rate_1m,report_weight_open_rate_3m,report_weight_open_rate_6m,report_weight_open_rate_12m,report_weight_open_rate_24m,report_wo_period_weight_open_rate,init_total_generation,row_count,share_period_plan,max_total_generation,max_SER_dinamic
0,2023-08,1.0,0,NaN,3,25.0,2006-06,2024-06,1.0,2023-08-31,5.224937e+01,8.292750e+00,0.010000,1.768259e+05,1.768259e+05,1.0,1.0,18.999929,2006-06_2024-06_1.0_25.0_1.0_3_0_RUR,1.50,1.50,0.000008,0.000008,1.768244e+05,1.768244e+05,RUR,NaN,NaN,NaN,NaN,NaN,NaN,1.768053e+05,15,0.953612,1.768259e+05,0.000008
1,2023-08,4.0,0,NaN,3,37.0,2009-06,2024-06,1.0,2023-08-31,3.592989e+02,5.795143e+01,0.010000,1.391181e+06,1.391181e+06,1.0,1.0,16.000000,2009-06_2024-06_4.0_37.0_1.0_3_0_RUR,11.82,11.82,0.000008,0.000008,1.391169e+06,1.391169e+06,RUR,NaN,NaN,NaN,NaN,NaN,NaN,1.390880e+06,27,0.944333,1.391181e+06,0.000008
2,2023-08,4.0,0,NaN,3,19.0,2009-09,2024-10,1.0,2023-08-31,4.411288e+02,7.114980e+01,0.010000,1.670922e+06,1.670922e+06,2.0,1.0,8.000000,2009-09_2024-10_4.0_19.0_1.0_3_0_RUR,14.19,14.19,0.000008,0.000008,1.670908e+06,1.670908e+06,RUR,NaN,NaN,NaN,NaN,NaN,NaN,1.670866e+06,5,0.922490,1.670922e+06,0.000008
3,2023-08,1.0,0,NaN,3,14.0,2012-08,2023-11,1.0,2023-08-31,8.467030e+00,1.365650e+00,0.010000,3.278378e+04,3.278378e+04,2.0,1.0,30.000000,2012-08_2023-11_1.0_14.0_1.0_3_0_RUR,0.27,0.27,0.000008,0.000008,3.278351e+04,3.278351e+04,RUR,NaN,NaN,NaN,NaN,NaN,NaN,3.278110e+04,11,0.977610,3.278378e+04,0.000008
4,2023-08,1.0,1,NaN,3,25.0,2014-07,2024-07,4.0,2023-08-31,2.635000e+03,4.250000e+02,2.500000,3.091008e+04,3.091008e+04,4.0,1.0,21.000000,2014-07_2024-07_1.0_25.0_4.0_3_1_RUR,65.49,65.49,0.002123,0.002123,3.084459e+04,3.084459e+04,RUR,NaN,NaN,NaN,NaN,NaN,NaN,3.046495e+04,8,0.908295,3.091008e+04,0.002123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9265,2023-08,3.0,0,1.0,3,4.0,2023-05,2023-11,7.0,2023-08-31,1.930961e+06,1.179322e+06,6.686535,5.764535e+07,5.764535e+07,1.0,1.0,24.261792,2023-05_2023-11_3.0_4.0_7.0_3_0_1.0_RUR,NaN,NaN,NaN,NaN,NaN,NaN,RUR,NaN,NaN,NaN,NaN,NaN,NaN,5.764535e+07,1,0.500000,5.764535e+07,NaN
9266,2023-08,2.0,0,100000.0,0,4.0,2023-07,2023-09,6.0,2023-08-31,3.416020e+04,6.658952e+04,5.744297,3.568082e+06,3.568082e+06,0.0,1.0,30.000000,2023-07_2023-09_2.0_4.0_6.0_0_0_100000.0_RUR,0.00,0.00,0.000000,0.000000,3.568082e+06,3.568082e+06,RUR,NaN,NaN,NaN,NaN,NaN,NaN,3.568082e+06,2,0.500000,3.568082e+06,0.000000
9267,2023-08,3.0,0,0.0,0,25.0,2023-07,2025-07,9.0,2023-08-31,1.364274e+06,3.095942e+06,8.387017,9.286719e+07,9.284955e+07,0.0,1.0,14.385989,2023-07_2025-07_3.0_25.0_9.0_0_0_0.0_RUR,-4717379.54,-4735014.72,-0.048341,-0.048522,9.758457e+07,9.758457e+07,RUR,NaN,9.412532,8.596218,8.001829,8.397441,8.974887,9.758457e+07,2,0.042408,9.286719e+07,-0.048341
9268,2023-08,5.0,0,0.0,0,37.0,2023-07,2026-07,10.0,2023-08-31,2.307131e+06,3.473577e+06,9.500000,9.911352e+07,9.911352e+07,0.0,1.0,8.033678,2023-07_2026-07_5.0_37.0_10.0_0_0_0.0_RUR,-68350557.42,-68350557.42,-0.408151,-0.408151,1.674641e+08,1.674641e+08,RUR,10.9,10.362142,9.167717,8.419679,8.812843,10.033960,1.674641e+08,2,0.028285,9.911352e+07,-0.408151


In [35]:
port_f = res_port[0]


for i in range(1, len(res_port)):
#     print(i)
    port_f = port_f.append(res_port[i])

In [36]:
port_f.groupby('report_month').sum()['total_generation'] / 10**9

report_month
2022-12    2825.966572
2023-01    2950.906502
2023-02    3000.235944
2023-03    3044.368600
2023-04    3077.629484
2023-05    3131.639969
2023-06    3148.000121
2023-07    3150.017358
2023-08    3295.017210
2023-09    3469.477693
Name: total_generation, dtype: float64

### Продолжаем расчеты

In [37]:
# если хотим обучить модели

from core.models.utils import run_spark_session
#spark = run_spark_session('check_calc')

spark = None #если без обучения

In [38]:
#train_end = '2023-02-28'

In [39]:
folder = '../data/trained_models'

sqlite_filepath = os.path.join(folder, 'modeldb_test.bin')

DB_URL = f"sqlite:///{sqlite_filepath}"
model_db = ModelDB(DB_URL)


In [40]:
scenario_df1

,expected_amount,SBER_max_rate,rate_sa_weighted,VTB_ftp_rate_[90d],VTB_ftp_rate_[180d],VTB_ftp_rate_[365d],VTB_ftp_rate_[548d],VTB_ftp_rate_[730d],VTB_ftp_rate_[1095d],VTB_weighted_rate_[mass]_[r0s0]_[90d],VTB_weighted_rate_[mass]_[r0s0]_[180d],VTB_weighted_rate_[mass]_[r0s0]_[365d],VTB_weighted_rate_[mass]_[r0s0]_[548d],VTB_weighted_rate_[mass]_[r0s0]_[730d],VTB_weighted_rate_[mass]_[r0s0]_[1095d],VTB_weighted_rate_[mass]_[r0s1]_[90d],VTB_weighted_rate_[mass]_[r0s1]_[180d],VTB_weighted_rate_[mass]_[r0s1]_[365d],VTB_weighted_rate_[mass]_[r0s1]_[548d],VTB_weighted_rate_[mass]_[r0s1]_[730d],VTB_weighted_rate_[mass]_[r0s1]_[1095d],VTB_weighted_rate_[mass]_[r1s0]_[90d],VTB_weighted_rate_[mass]_[r1s0]_[180d],VTB_weighted_rate_[mass]_[r1s0]_[365d],VTB_weighted_rate_[mass]_[r1s0]_[548d],VTB_weighted_rate_[mass]_[r1s0]_[730d],VTB_weighted_rate_[mass]_[r1s0]_[1095d],VTB_weighted_rate_[mass]_[r1s1]_[90d],VTB_weighted_rate_[mass]_[r1s1]_[180d],VTB_weighted_rate_[mass]_[r1s1]_[365d],VTB_weighted_rate_[mass]_[r1s1]_[548d],VTB_weighted_rate_[mass]_[r1s1]_[730d],VTB_weighted_rate_[mass]_[r1s1]_[1095d],VTB_weighted_rate_[priv]_[r0s0]_[90d],VTB_weighted_rate_[priv]_[r0s0]_[180d],VTB_weighted_rate_[priv]_[r0s0]_[365d],VTB_weighted_rate_[priv]_[r0s0]_[548d],VTB_weighted_rate_[priv]_[r0s0]_[730d],VTB_weighted_rate_[priv]_[r0s0]_[1095d],VTB_weighted_rate_[priv]_[r0s1]_[90d],VTB_weighted_rate_[priv]_[r0s1]_[180d],VTB_weighted_rate_[priv]_[r0s1]_[365d],VTB_weighted_rate_[priv]_[r0s1]_[548d],VTB_weighted_rate_[priv]_[r0s1]_[730d],VTB_weighted_rate_[priv]_[r0s1]_[1095d],VTB_weighted_rate_[priv]_[r1s0]_[90d],VTB_weighted_rate_[priv]_[r1s0]_[180d],VTB_weighted_rate_[priv]_[r1s0]_[365d],VTB_weighted_rate_[priv]_[r1s0]_[548d],VTB_weighted_rate_[priv]_[r1s0]_[730d],VTB_weighted_rate_[priv]_[r1s0]_[1095d],VTB_weighted_rate_[priv]_[r1s1]_[90d],VTB_weighted_rate_[priv]_[r1s1]_[180d],VTB_weighted_rate_[priv]_[r1s1]_[365d],VTB_weighted_rate_[priv]_[r1s1]_[548d],VTB_weighted_rate_[priv]_[r1s1]_[730d],VTB_weighted_rate_[priv]_[r1s1]_[1095d],VTB_weighted_rate_[vip]_[r0s0]_[90d],VTB_weighted_rate_[vip]_[r0s0]_[180d],VTB_weighted_rate_[vip]_[r0s0]_[365d],VTB_weighted_rate_[vip]_[r0s0]_[548d],VTB_weighted_rate_[vip]_[r0s0]_[730d],VTB_weighted_rate_[vip]_[r0s0]_[1095d],VTB_weighted_rate_[vip]_[r0s1]_[90d],VTB_weighted_rate_[vip]_[r0s1]_[180d],VTB_weighted_rate_[vip]_[r0s1]_[365d],VTB_weighted_rate_[vip]_[r0s1]_[548d],VTB_weighted_rate_[vip]_[r0s1]_[730d],VTB_weighted_rate_[vip]_[r0s1]_[1095d],VTB_weighted_rate_[vip]_[r1s0]_[90d],VTB_weighted_rate_[vip]_[r1s0]_[180d],VTB_weighted_rate_[vip]_[r1s0]_[365d],VTB_weighted_rate_[vip]_[r1s0]_[548d],VTB_weighted_rate_[vip]_[r1s0]_[730d],VTB_weighted_rate_[vip]_[r1s0]_[1095d],VTB_weighted_rate_[vip]_[r1s1]_[90d],VTB_weighted_rate_[vip]_[r1s1]_[180d],VTB_weighted_rate_[vip]_[r1s1]_[365d],VTB_weighted_rate_[vip]_[r1s1]_[548d],VTB_weighted_rate_[vip]_[r1s1]_[730d],VTB_weighted_rate_[vip]_[r1s1]_[1095d]
2023-01-31,NaN,8.000000,5.003411,8.000000,8.250000,8.550000,8.850000,9.150000,9.500000,6.999905,7.518473,7.471891,5.660102,7.578739,8.390000,0.01,0.01,0.01,0.01,0.01,0.01,5.842649,3.794318,3.268132,2.985803,2.999146,2.994811,6.029594,6.186708,5.133139,3.150000,4.504944,2.000000,7.000000,7.440814,7.551823,6.000000,7.823649,8.390000,0.01,0.01,0.01,0.01,0.01,0.01,6.552139,6.231580,5.474193,2.946051,2.924987,2.909715,6.136634,6.250000,4.750000,3.75,4.50,2.000000,7.000000,7.398181,7.620515,7.842370,8.200000,8.397031,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,6.220162,6.467566,6.175783,5.400000,5.528164,5.400000
2023-02-28,NaN,9.500000,5.047190,8.005556,8.255556,8.550000,8.850000,9.150000,9.500000,6.951340,7.439611,7.457628,5.830272,7.578739,8.390000,0.01,0.01,0.01,0.01,0.01,0.01,5.842649,3.794318,3.268132,2.985803,2.999146,2.994811,6.026958,6.326632,6.113943,3.150000,4.500000,2.000000,6.991730,7.433784,7.554141,5.113112,7.823649,8.390000,0.01,0.01,0.01,0.01,0.01,0.01,6.552139,6.231580,5.474193,2.946051,2.924987,2.909715,

### Старт расчетов

In [41]:
def calc_res(train_end, scenario_df, model_db):
    
    port_folder = '../data/portfolio_data'
    folder = '../data/trained_models'
    portfolio = load_portfolio(train_end, port_folder)
    spark = None
    
    config: ForecastConfig = ForecastConfig(
    first_train_end_dt = train_end,
    horizon = horizon,
    trainers = DepositModels.trainers,
    data_loaders = DepositModels.dataloaders,
    calculator_type = DepositIterativeCalculator,
    calc_type = DepositsCalculationType,
    adapter_types = DepositModels.adapter_types,
    scenario_data = scenario_df,
    portfolio = portfolio
    )

    training_manager = TrainingManager(spark, config.trainers, folder, model_db)   
    engine: ForecastEngine = ForecastEngine(spark, config, training_manager, overwrite_models=False)
        
    engine.run_all()
    
    agg_res = engine.calc_results['Deposits']['agg_data']
    
    res = agg_res.groupby('report_dt').sum()
    
    return res

In [42]:
# Дата из который мы прогнозируем (пока что не меняется)
train_end = datetime(year=2023, month=1, day=31)
train_end

datetime.datetime(2023, 1, 31, 0, 0)

In [43]:
(train_end + MonthEnd(1)).to_pydatetime()

datetime.datetime(2023, 2, 28, 0, 0)

In [44]:
# train_dt = (date.today() + MonthEnd(-11)).to_pydatetime()
# train_dt

In [45]:
scenario_df

,expected_amount,SBER_max_rate,rate_sa_weighted,VTB_ftp_rate_[90d],VTB_ftp_rate_[180d],VTB_ftp_rate_[365d],VTB_ftp_rate_[548d],VTB_ftp_rate_[730d],VTB_ftp_rate_[1095d],VTB_weighted_rate_[mass]_[r0s0]_[90d],VTB_weighted_rate_[mass]_[r0s0]_[180d],VTB_weighted_rate_[mass]_[r0s0]_[365d],VTB_weighted_rate_[mass]_[r0s0]_[548d],VTB_weighted_rate_[mass]_[r0s0]_[730d],VTB_weighted_rate_[mass]_[r0s0]_[1095d],VTB_weighted_rate_[mass]_[r0s1]_[90d],VTB_weighted_rate_[mass]_[r0s1]_[180d],VTB_weighted_rate_[mass]_[r0s1]_[365d],VTB_weighted_rate_[mass]_[r0s1]_[548d],VTB_weighted_rate_[mass]_[r0s1]_[730d],VTB_weighted_rate_[mass]_[r0s1]_[1095d],VTB_weighted_rate_[mass]_[r1s0]_[90d],VTB_weighted_rate_[mass]_[r1s0]_[180d],VTB_weighted_rate_[mass]_[r1s0]_[365d],VTB_weighted_rate_[mass]_[r1s0]_[548d],VTB_weighted_rate_[mass]_[r1s0]_[730d],VTB_weighted_rate_[mass]_[r1s0]_[1095d],VTB_weighted_rate_[mass]_[r1s1]_[90d],VTB_weighted_rate_[mass]_[r1s1]_[180d],VTB_weighted_rate_[mass]_[r1s1]_[365d],VTB_weighted_rate_[mass]_[r1s1]_[548d],VTB_weighted_rate_[mass]_[r1s1]_[730d],VTB_weighted_rate_[mass]_[r1s1]_[1095d],VTB_weighted_rate_[priv]_[r0s0]_[90d],VTB_weighted_rate_[priv]_[r0s0]_[180d],VTB_weighted_rate_[priv]_[r0s0]_[365d],VTB_weighted_rate_[priv]_[r0s0]_[548d],VTB_weighted_rate_[priv]_[r0s0]_[730d],VTB_weighted_rate_[priv]_[r0s0]_[1095d],VTB_weighted_rate_[priv]_[r0s1]_[90d],VTB_weighted_rate_[priv]_[r0s1]_[180d],VTB_weighted_rate_[priv]_[r0s1]_[365d],VTB_weighted_rate_[priv]_[r0s1]_[548d],VTB_weighted_rate_[priv]_[r0s1]_[730d],VTB_weighted_rate_[priv]_[r0s1]_[1095d],VTB_weighted_rate_[priv]_[r1s0]_[90d],VTB_weighted_rate_[priv]_[r1s0]_[180d],VTB_weighted_rate_[priv]_[r1s0]_[365d],VTB_weighted_rate_[priv]_[r1s0]_[548d],VTB_weighted_rate_[priv]_[r1s0]_[730d],VTB_weighted_rate_[priv]_[r1s0]_[1095d],VTB_weighted_rate_[priv]_[r1s1]_[90d],VTB_weighted_rate_[priv]_[r1s1]_[180d],VTB_weighted_rate_[priv]_[r1s1]_[365d],VTB_weighted_rate_[priv]_[r1s1]_[548d],VTB_weighted_rate_[priv]_[r1s1]_[730d],VTB_weighted_rate_[priv]_[r1s1]_[1095d],VTB_weighted_rate_[vip]_[r0s0]_[90d],VTB_weighted_rate_[vip]_[r0s0]_[180d],VTB_weighted_rate_[vip]_[r0s0]_[365d],VTB_weighted_rate_[vip]_[r0s0]_[548d],VTB_weighted_rate_[vip]_[r0s0]_[730d],VTB_weighted_rate_[vip]_[r0s0]_[1095d],VTB_weighted_rate_[vip]_[r0s1]_[90d],VTB_weighted_rate_[vip]_[r0s1]_[180d],VTB_weighted_rate_[vip]_[r0s1]_[365d],VTB_weighted_rate_[vip]_[r0s1]_[548d],VTB_weighted_rate_[vip]_[r0s1]_[730d],VTB_weighted_rate_[vip]_[r0s1]_[1095d],VTB_weighted_rate_[vip]_[r1s0]_[90d],VTB_weighted_rate_[vip]_[r1s0]_[180d],VTB_weighted_rate_[vip]_[r1s0]_[365d],VTB_weighted_rate_[vip]_[r1s0]_[548d],VTB_weighted_rate_[vip]_[r1s0]_[730d],VTB_weighted_rate_[vip]_[r1s0]_[1095d],VTB_weighted_rate_[vip]_[r1s1]_[90d],VTB_weighted_rate_[vip]_[r1s1]_[180d],VTB_weighted_rate_[vip]_[r1s1]_[365d],VTB_weighted_rate_[vip]_[r1s1]_[548d],VTB_weighted_rate_[vip]_[r1s1]_[730d],VTB_weighted_rate_[vip]_[r1s1]_[1095d]
2023-01-31,NaN,11.2,5.0,12.3,12.0,12.0,12.0,12.0,12.0,10.3665,10.08,10.08,10.08,9.98,9.98,9.3665,9.08,9.08,9.08,8.98,8.98,9.3665,9.08,9.08,9.08,8.98,8.98,9.1665,8.88,8.88,8.88,8.78,8.78,10.7665,10.48,10.48,10.48,10.38,10.38,9.7665,9.48,9.48,9.48,9.38,9.38,9.7665,9.48,9.48,9.48,9.38,9.38,9.5665,9.28,9.28,9.28,9.18,9.18,11.1665,10.88,10.88,10.88,10.78,10.78,10.1665,9.88,9.88,9.88,9.78,9.78,10.1665,9.88,9.88,9.88,9.78,9.78,9.9665,9.68,9.68,9.68,9.58,9.58
2023-02-28,NaN,11.2,5.0,12.3,12.0,12.0,12.0,12.0,12.0,10.3665,10.08,10.08,10.08,9.98,9.98,9.3665,9.08,9.08,9.08,8.98,8.98,9.3665,9.08,9.08,9.08,8.98,8.98,9.1665,8.88,8.88,8.88,8.78,8.78,10.7665,10.48,10.48,10.48,10.38,10.38,9.7665,9.48,9.48,9.48,9.38,9.38,9.7665,9.48,9.48,9.48,9.38,9.38,9.5665,9.28,9.28,9.28,9.18,9.18,11.1665,10.88,10.88,10.88,10.78,10.78,10.1665,9.88,9.88,9.88,9.78,9.78,10.1665,9.88,9.88,9.88,9.78,9.78,9.9665,9.68,9.68,9.68,9.58,9.58
2023-03-31,NaN,11.2,5.0,12.3,12.0,12.0,12.0,12.0,12.0,10.3665,10.08,10.08,10.08,9.98,9.98,9.3665,9.08,9.08,9.08,8.98,8.98,9.3665,9.08,9.

In [46]:
horizon = 3

In [51]:
res_list = []

for i in range(0,9):
    train_dt = (train_end + MonthEnd(i)).to_pydatetime()
    train_dt_end = (train_dt + MonthEnd(3)).to_pydatetime()
    scenario_df = scenario_df1[(scenario_df1.index>=(train_dt+MonthEnd(1)))].head(3)
    
    
    r = calc_res(train_dt, scenario_df, model_db)
    
    res_list.append(r)


INFO:core:missing models: []
INFO:core:add_models_from_bytes
INFO:core:plan_close_201402_202301 - adapter <class 'core.models.plan_close.plan_close_model.PlanCloseModelAdapter'>
INFO:core:renewal_model_201401_202301 - adapter <class 'core.models.renewal.renewal_model.RenewalModelAdapter'>
INFO:core:maturity_structure_mass_r0s0_201401_202301 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR0S0ModelAdapter'>
INFO:core:maturity_structure_mass_r0s1_202001_202301 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR0S1ModelAdapter'>
INFO:core:maturity_structure_mass_r1s0_201401_202301 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR1S0ModelAdapter'>
INFO:core:maturity_structure_mass_r1s1_201401_202301 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStruct

In [62]:
res_list[0]

,replenishable_flg,subtraction_flg,month_maturity,target_maturity_days,start_balance,balance_gain,balance,newbusiness,contract_close,early_withdrawal,operations,interests,renewal,universal_weight_id
report_dt,,,,,,,,,,,,,,
2023-02-28,32,18,873,26532,2.950907e+12,-2.844421e+11,2.666464e+12,1.321094e+11,-4.469379e+11,-1.371197e+10,2.945957e+10,1.463879e+10,7.080551e+10,251
2023-03-31,32,18,873,26532,2.666464e+12,-1.692758e+11,2.497188e+12,2.054769e+11,-4.146255e+11,-1.115682e+10,3.795759e+10,1.307192e+10,1.106811e+11,251
2023-04-30,32,18,873,26532,2.497188e+12,-8.024411e+10,2.416944e+12,1.394209e+11,-2.614340e+11,-1.207087e+10,4.059940e+10,1.324050e+10,3.986056e+10,251


In [57]:
df = pd.DataFrame(index=scenario_df1.index)

for i in range(len(res_list)):
    tmp = res_list[i]
    name = tmp.index[0]
    df[name] = tmp['balance']

In [60]:
# для записи и чтения экселя
import pip
pip.main(['install', 'openpyxl'])
import openpyxl

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/250.0 kB ? eta -:--:--

Collecting et-xmlfile

Downloading http://nxrm:8081/repository/pypi/packages/et-xmlfile/1.1.0/et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)

Installing collected packages: et-xmlfile, openpyxl

Successfully installed et-xmlfile-1.1.0 openpyxl-3.1.2

In [61]:
df.to_excel('res_pred.xlsx')

In [48]:
scenario_df1[(scenario_df1.index>=(train_dt+MonthEnd(1)))].head(3)

,expected_amount,SBER_max_rate,rate_sa_weighted,VTB_ftp_rate_[90d],VTB_ftp_rate_[180d],VTB_ftp_rate_[365d],VTB_ftp_rate_[548d],VTB_ftp_rate_[730d],VTB_ftp_rate_[1095d],VTB_weighted_rate_[mass]_[r0s0]_[90d],VTB_weighted_rate_[mass]_[r0s0]_[180d],VTB_weighted_rate_[mass]_[r0s0]_[365d],VTB_weighted_rate_[mass]_[r0s0]_[548d],VTB_weighted_rate_[mass]_[r0s0]_[730d],VTB_weighted_rate_[mass]_[r0s0]_[1095d],VTB_weighted_rate_[mass]_[r0s1]_[90d],VTB_weighted_rate_[mass]_[r0s1]_[180d],VTB_weighted_rate_[mass]_[r0s1]_[365d],VTB_weighted_rate_[mass]_[r0s1]_[548d],VTB_weighted_rate_[mass]_[r0s1]_[730d],VTB_weighted_rate_[mass]_[r0s1]_[1095d],VTB_weighted_rate_[mass]_[r1s0]_[90d],VTB_weighted_rate_[mass]_[r1s0]_[180d],VTB_weighted_rate_[mass]_[r1s0]_[365d],VTB_weighted_rate_[mass]_[r1s0]_[548d],VTB_weighted_rate_[mass]_[r1s0]_[730d],VTB_weighted_rate_[mass]_[r1s0]_[1095d],VTB_weighted_rate_[mass]_[r1s1]_[90d],VTB_weighted_rate_[mass]_[r1s1]_[180d],VTB_weighted_rate_[mass]_[r1s1]_[365d],VTB_weighted_rate_[mass]_[r1s1]_[548d],VTB_weighted_rate_[mass]_[r1s1]_[730d],VTB_weighted_rate_[mass]_[r1s1]_[1095d],VTB_weighted_rate_[priv]_[r0s0]_[90d],VTB_weighted_rate_[priv]_[r0s0]_[180d],VTB_weighted_rate_[priv]_[r0s0]_[365d],VTB_weighted_rate_[priv]_[r0s0]_[548d],VTB_weighted_rate_[priv]_[r0s0]_[730d],VTB_weighted_rate_[priv]_[r0s0]_[1095d],VTB_weighted_rate_[priv]_[r0s1]_[90d],VTB_weighted_rate_[priv]_[r0s1]_[180d],VTB_weighted_rate_[priv]_[r0s1]_[365d],VTB_weighted_rate_[priv]_[r0s1]_[548d],VTB_weighted_rate_[priv]_[r0s1]_[730d],VTB_weighted_rate_[priv]_[r0s1]_[1095d],VTB_weighted_rate_[priv]_[r1s0]_[90d],VTB_weighted_rate_[priv]_[r1s0]_[180d],VTB_weighted_rate_[priv]_[r1s0]_[365d],VTB_weighted_rate_[priv]_[r1s0]_[548d],VTB_weighted_rate_[priv]_[r1s0]_[730d],VTB_weighted_rate_[priv]_[r1s0]_[1095d],VTB_weighted_rate_[priv]_[r1s1]_[90d],VTB_weighted_rate_[priv]_[r1s1]_[180d],VTB_weighted_rate_[priv]_[r1s1]_[365d],VTB_weighted_rate_[priv]_[r1s1]_[548d],VTB_weighted_rate_[priv]_[r1s1]_[730d],VTB_weighted_rate_[priv]_[r1s1]_[1095d],VTB_weighted_rate_[vip]_[r0s0]_[90d],VTB_weighted_rate_[vip]_[r0s0]_[180d],VTB_weighted_rate_[vip]_[r0s0]_[365d],VTB_weighted_rate_[vip]_[r0s0]_[548d],VTB_weighted_rate_[vip]_[r0s0]_[730d],VTB_weighted_rate_[vip]_[r0s0]_[1095d],VTB_weighted_rate_[vip]_[r0s1]_[90d],VTB_weighted_rate_[vip]_[r0s1]_[180d],VTB_weighted_rate_[vip]_[r0s1]_[365d],VTB_weighted_rate_[vip]_[r0s1]_[548d],VTB_weighted_rate_[vip]_[r0s1]_[730d],VTB_weighted_rate_[vip]_[r0s1]_[1095d],VTB_weighted_rate_[vip]_[r1s0]_[90d],VTB_weighted_rate_[vip]_[r1s0]_[180d],VTB_weighted_rate_[vip]_[r1s0]_[365d],VTB_weighted_rate_[vip]_[r1s0]_[548d],VTB_weighted_rate_[vip]_[r1s0]_[730d],VTB_weighted_rate_[vip]_[r1s0]_[1095d],VTB_weighted_rate_[vip]_[r1s1]_[90d],VTB_weighted_rate_[vip]_[r1s1]_[180d],VTB_weighted_rate_[vip]_[r1s1]_[365d],VTB_weighted_rate_[vip]_[r1s1]_[548d],VTB_weighted_rate_[vip]_[r1s1]_[730d],VTB_weighted_rate_[vip]_[r1s1]_[1095d]
2023-10-31,NaN,12.0,7.0,12.257143,12.0,12.0,12.0,12.0,12.0,9.95961,9.105403,7.833557,7.984999,8.023832,9.797095,0.01,0.01,0.01,0.01,0.01,0.01,7.368173,6.115845,3.98016,2.978357,2.983666,2.976729,7.640647,6.425692,6.483047,2.0,2.0,7.942598,10.570646,9.415825,8.19313,8.254452,8.368374,9.980873,0.01,0.01,0.01,0.01,0.01,0.01,8.385679,8.958574,5.999389,2.946051,3.0,2.922613,8.289371,3.686667,3.0,2.0,2.0,12.12219,10.893834,9.830158,8.680745,8.494777,8.519724,10.348778,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,9.657904,11.814359,12.063286,5.195343,5.451624,12.106866
2023-11-30,NaN,12.0,7.0,12.257143,12.0,12.0,12.0,12.0,12.0,9.95961,9.105403,7.833557,7.984999,8.023832,9.797095,0.01,0.01,0.01,0.01,0.01,0.01,7.368173,6.115845,3.98016,2.978357,2.983666,2.976729,7.640647,6.425692,6.483047,2.0,2.0,7.942598,10.570646,9.415825,8.19313,8.254452,8.368374,9.980873,0.01,0.01,0.01,0.01,0.01,0.01,8.385679,8.958574,5.999389,2.946051,3.0,2.922613,8.289371,3.686667,3.0,2.0,2.0,12.12219,10.893834,9.830158,8.680745,8.494777,8.519724,10.3487

In [63]:
scenario_df1

,expected_amount,SBER_max_rate,rate_sa_weighted,VTB_ftp_rate_[90d],VTB_ftp_rate_[180d],VTB_ftp_rate_[365d],VTB_ftp_rate_[548d],VTB_ftp_rate_[730d],VTB_ftp_rate_[1095d],VTB_weighted_rate_[mass]_[r0s0]_[90d],VTB_weighted_rate_[mass]_[r0s0]_[180d],VTB_weighted_rate_[mass]_[r0s0]_[365d],VTB_weighted_rate_[mass]_[r0s0]_[548d],VTB_weighted_rate_[mass]_[r0s0]_[730d],VTB_weighted_rate_[mass]_[r0s0]_[1095d],VTB_weighted_rate_[mass]_[r0s1]_[90d],VTB_weighted_rate_[mass]_[r0s1]_[180d],VTB_weighted_rate_[mass]_[r0s1]_[365d],VTB_weighted_rate_[mass]_[r0s1]_[548d],VTB_weighted_rate_[mass]_[r0s1]_[730d],VTB_weighted_rate_[mass]_[r0s1]_[1095d],VTB_weighted_rate_[mass]_[r1s0]_[90d],VTB_weighted_rate_[mass]_[r1s0]_[180d],VTB_weighted_rate_[mass]_[r1s0]_[365d],VTB_weighted_rate_[mass]_[r1s0]_[548d],VTB_weighted_rate_[mass]_[r1s0]_[730d],VTB_weighted_rate_[mass]_[r1s0]_[1095d],VTB_weighted_rate_[mass]_[r1s1]_[90d],VTB_weighted_rate_[mass]_[r1s1]_[180d],VTB_weighted_rate_[mass]_[r1s1]_[365d],VTB_weighted_rate_[mass]_[r1s1]_[548d],VTB_weighted_rate_[mass]_[r1s1]_[730d],VTB_weighted_rate_[mass]_[r1s1]_[1095d],VTB_weighted_rate_[priv]_[r0s0]_[90d],VTB_weighted_rate_[priv]_[r0s0]_[180d],VTB_weighted_rate_[priv]_[r0s0]_[365d],VTB_weighted_rate_[priv]_[r0s0]_[548d],VTB_weighted_rate_[priv]_[r0s0]_[730d],VTB_weighted_rate_[priv]_[r0s0]_[1095d],VTB_weighted_rate_[priv]_[r0s1]_[90d],VTB_weighted_rate_[priv]_[r0s1]_[180d],VTB_weighted_rate_[priv]_[r0s1]_[365d],VTB_weighted_rate_[priv]_[r0s1]_[548d],VTB_weighted_rate_[priv]_[r0s1]_[730d],VTB_weighted_rate_[priv]_[r0s1]_[1095d],VTB_weighted_rate_[priv]_[r1s0]_[90d],VTB_weighted_rate_[priv]_[r1s0]_[180d],VTB_weighted_rate_[priv]_[r1s0]_[365d],VTB_weighted_rate_[priv]_[r1s0]_[548d],VTB_weighted_rate_[priv]_[r1s0]_[730d],VTB_weighted_rate_[priv]_[r1s0]_[1095d],VTB_weighted_rate_[priv]_[r1s1]_[90d],VTB_weighted_rate_[priv]_[r1s1]_[180d],VTB_weighted_rate_[priv]_[r1s1]_[365d],VTB_weighted_rate_[priv]_[r1s1]_[548d],VTB_weighted_rate_[priv]_[r1s1]_[730d],VTB_weighted_rate_[priv]_[r1s1]_[1095d],VTB_weighted_rate_[vip]_[r0s0]_[90d],VTB_weighted_rate_[vip]_[r0s0]_[180d],VTB_weighted_rate_[vip]_[r0s0]_[365d],VTB_weighted_rate_[vip]_[r0s0]_[548d],VTB_weighted_rate_[vip]_[r0s0]_[730d],VTB_weighted_rate_[vip]_[r0s0]_[1095d],VTB_weighted_rate_[vip]_[r0s1]_[90d],VTB_weighted_rate_[vip]_[r0s1]_[180d],VTB_weighted_rate_[vip]_[r0s1]_[365d],VTB_weighted_rate_[vip]_[r0s1]_[548d],VTB_weighted_rate_[vip]_[r0s1]_[730d],VTB_weighted_rate_[vip]_[r0s1]_[1095d],VTB_weighted_rate_[vip]_[r1s0]_[90d],VTB_weighted_rate_[vip]_[r1s0]_[180d],VTB_weighted_rate_[vip]_[r1s0]_[365d],VTB_weighted_rate_[vip]_[r1s0]_[548d],VTB_weighted_rate_[vip]_[r1s0]_[730d],VTB_weighted_rate_[vip]_[r1s0]_[1095d],VTB_weighted_rate_[vip]_[r1s1]_[90d],VTB_weighted_rate_[vip]_[r1s1]_[180d],VTB_weighted_rate_[vip]_[r1s1]_[365d],VTB_weighted_rate_[vip]_[r1s1]_[548d],VTB_weighted_rate_[vip]_[r1s1]_[730d],VTB_weighted_rate_[vip]_[r1s1]_[1095d]
2023-01-31,NaN,8.000000,5.003411,8.000000,8.250000,8.550000,8.850000,9.150000,9.500000,6.999905,7.518473,7.471891,5.660102,7.578739,8.390000,0.01,0.01,0.01,0.01,0.01,0.01,5.842649,3.794318,3.268132,2.985803,2.999146,2.994811,6.029594,6.186708,5.133139,3.150000,4.504944,2.000000,7.000000,7.440814,7.551823,6.000000,7.823649,8.390000,0.01,0.01,0.01,0.01,0.01,0.01,6.552139,6.231580,5.474193,2.946051,2.924987,2.909715,6.136634,6.250000,4.750000,3.75,4.50,2.000000,7.000000,7.398181,7.620515,7.842370,8.200000,8.397031,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,6.220162,6.467566,6.175783,5.400000,5.528164,5.400000
2023-02-28,NaN,9.500000,5.047190,8.005556,8.255556,8.550000,8.850000,9.150000,9.500000,6.951340,7.439611,7.457628,5.830272,7.578739,8.390000,0.01,0.01,0.01,0.01,0.01,0.01,5.842649,3.794318,3.268132,2.985803,2.999146,2.994811,6.026958,6.326632,6.113943,3.150000,4.500000,2.000000,6.991730,7.433784,7.554141,5.113112,7.823649,8.390000,0.01,0.01,0.01,0.01,0.01,0.01,6.552139,6.231580,5.474193,2.946051,2.924987,2.909715,

In [49]:
# вывод
portfolio_res = engine.calc_results['Deposits']['portfolio'] 
agg_res = engine.calc_results['Deposits']['agg_data']
maturity = engine.calc_results['Deposits']['maturity']
CurrentAccounts = engine.calc_results['CurrentAccounts']
SavingAccounts = engine.calc_results['SavingAccounts']
volumes = engine.calc_results['Volumes']

NameError: name 'engine' is not defined

In [ ]:
agg_res.groupby('report_dt').sum()

In [ ]:
# для записи и чтения экселя
# import pip
# pip.main(['install', 'openpyxl'])
# import openpyxl

In [ ]:
# import openpyxl

In [ ]:
# with pd.ExcelWriter("august_res_v2.xlsx") as writer:
#     portfolio_res.to_excel(writer, sheet_name='portfolio_res', index=False)
#     agg_res.to_excel(writer, sheet_name='agg_res', index=False)
#     maturity.to_excel(writer, sheet_name='maturity', index=False)
#     CurrentAccounts.to_excel(writer, sheet_name='CurrentAccounts', index=False)
#     SavingAccounts.to_excel(writer, sheet_name='SavingAccounts', index=False)
#     volumes.to_excel(writer, sheet_name='volumes', index=False)
#     pd.DataFrame(scenario_data).to_excel(writer, sheet_name='scenario', index=False)